In [ ]:
# !pip uninstall tflearn
!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing -q

  Preparing metadata (setup.py) ... done


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
from tensorflow.python.framework import ops
ops.reset_default_graph()
import random
nltk.download('punkt')

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import json
with open('intents.json') as data_file:
    data = json.load(data_file)

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append((intent['tag']))

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [ ]:
training = numpy.array(training)
output = numpy.array(output)

In [ ]:
len(output[0])

30

In [ ]:
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("/content/model.tflearn")

Training Step: 11999  | total loss: 0.00950 | time: 0.050s
| Adam | epoch: 1000 | loss: 0.00950 - acc: 0.9952 -- iter: 88/91
Training Step: 12000  | total loss: 0.01745 | time: 0.054s
| Adam | epoch: 1000 | loss: 0.01745 - acc: 0.9956 -- iter: 91/91
--


In [ ]:
model.load("/content/model.tflearn")

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)


def chat():
    bot_name = "boBot"
    print(f"Start talking with {bot_name} (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(f"{bot_name}: {random.choice(responses)}")

chat()

Start talking with boBot (type quit to stop)!
You: Hai
boBot: Terima kasih! Kami selalu berusaha memberikan yang terbaik.
You: Apa saja layanan yang tersedia?
boBot: Kami menawarkan berbagai layanan: 
 1. Konsultasi produk 
 2. Informasi ketersediaan stok 
 3. Bantuan teknis 
 4. Pembaruan pesanan 
 5. Pemesanan produk baru 
 6. Layanan pelanggan 24/7. Bagaimana saya bisa membantu Anda hari ini?
You: Buat saya tertawa
boBot: Kenapa komputer tidak bisa berbohong? Karena mereka selalu menyimpan catatan!
You: Siapa inspirasi anda?
boBot: Saya terinspirasi oleh tim pengembang saya yang selalu bekerja keras untuk memberikan layanan terbaik kepada Anda.
You: Halo
boBot: Selamat datang! Bagaimana saya bisa membantu Anda hari ini?
You: Dadah
boBot: Terima kasih sudah menghubungi kami. Sampai jumpa lagi!
You: quit
